#Описание подхода для разделения данных (спринт 3) (Команда № 17)

##I. Подготовка данных

### 1. Отбор авторов и статей (стратегия):

  1) В первую очередь мы отобрали статьи, на которых обучали модели кластеризации и классификации, а именно:
- по году публикации >= 1990
- по числу цитирований > 2

Для рекомендаций соавторов было решено использовать связи авторов по статьям, поэтому из данных удаляем все статьи с одним автором:
- по числу авторов > 1 (чтобы был соавтор)

### 2. Строим таблицу взаимодействия авторов по статьям (матрица А).

     Автор        Соавтор      Кол-во совместных публикаций
    auth_id_1   co_auth_id_1     article_number
    auth_id_1   co_auth_id_2     article_number
     ...           ...           ... 

    auth_id_N   co_auth_id_M     article_number

Здесь последний столбец - количество совместных публикаций конкретных автора и соавтора. Причем для каждой статьи будет по крайней мере 2 строки, отличающихся только следованием author_id и co_author_id. Если в статье количество авторов больше 2, то для каждой упорядоченной пары соавторов будет своя строка.

  2) В таблице А группируем значения по столбцу Автор и удаляем тех авторов, у которых количество соавторов меньше 3 (это число далее будет уточняться в процессе разработки модели рекомендаций, на основе тестовых характеристик). Это сделано для того, чтобы при разделении данных на трэйн и тест у нас не оказалось авторов с единственным взаимодействием, так как этого не достаточно для выроботки рекомендаций по соавторам.



### Подготовленные данные доступны по ссылкам:
**Train:** https://drive.google.com/file/d/1wNXHlb-Ti5gS4TUg1xE3AYb67ltXRwk-/view?usp=sharing

**Test:** https://drive.google.com/file/d/1cTa38EFTk1PVSN1dUzUJozO0lMxIk03O/view?usp=sharing


#II. Сплиттинг данных

 Были рассмотрены следующие стратегии разделения данных:

	a) случайное разделение:  для тестовой выборки выбираем случайным образом 20% данных;
	б) Leave-One-Last-Item: для каждого автора для тестовой выборки выбираем элемент с максимальным годом публикации (год можно дополнительно учитывать в А)
	в) temporal split: для каждого автора для тестовой выборки выбираем элементы, которые соответствуют публикациям после заданной даты (например после 2020)
	г) user-split: 20% случайных авторов отнести к тестовой выборке. 

  1) Так как "хорошая" рекомендация соавтора - это соавтор занимающийся исследованием схожей научной проблематики, и как правило ученые не меняют направление научной деятельности со временем, то мы отбросили методы с временными привязками Leave-One-Last-Item и temporal split. Кроме того, даже если наше предположение не верно, мы всегда можем сгруппировать наших авторов по тематикам с помощью обученой модели классификации и давать рекомендации в рамках одного класса. 
  
  2) Для тестовой выборки было решено использовать случайное разделение данных стратифицированное по количеству соавторов. Чтобы в обучающую и тестовую выборку попали данные примерно в одном и том же соотношении относительно количества соавторств. Анализ данных показал, что чаще всего число соавторов  ближе к 3-4-5, поэтому для них мы берем меньше интервалы. Авторы, у которых более 200 соавторов - большая редкость, поэтому интервал шире. Интервалы подбирались с учетом гистограмм распределения числа соавторов.

Кроме того, при разделении данных, учитывалось условие, чтобы какой-то автор не попал полностью в тестовую или обучающую выборку.
	


## III. Подготовка графового представления данных

Из таблицы **А** строим матрицу инцидентности **C** для графа взаимодействия авторов.

Отличием этой матрицы от "классической" матрицы инцидентности является только то, что на пересечении двух авторов будет стоять вес ребра:  0 (если у них нет совместных статей) или article_number (количество совместных статей). 

Матрица инцидентности **C** может быть построена с помощью pandas.pivot_table. В результате будем иметь разреженную матрицу размера $N \times N$, в которой большинство значений $val\_ij=0$:

$\begin{array}{|c|c|} \hline
authors\_id & author\_id\_1 & author\_id\_2 & \dots & author\_id\_N\\ \hline
author\_id\_1 & val\_11 & val\_12 & \dots & val\_1N\\
author\_id\_2 & val\_21 & val\_22 & \dots & val\_2N\\
author\_id\_3 & val\_31 & val\_32 & \dots & val\_3N\\
\dots & \dots & \dots & \dots & \dots\\
author\_id\_N & val\_N1 & 0 & \dots & val\_NN\\ \hline
\end{array}$


Кроме того для уменьшения размерности матрицы **С** можно построить набор матриц $C_1, C_2, ... , C_k$, где $k$ - число классов, для каждого класса в отдельности. Это позволит в дальнейшем ускорить поиск наилучшего соавтора внутри одного класса.

##IV. Метрики для оценки качества рекомендаций

Были рассмотрены следующие группы метрик для измерения качества рекомендаций:

1. Метрики сходства на основе векторов признаков авторов
  а) Косинусное сходство ( Cosine similarity)
Чтобы вычислить сходство между двумя авторами, мы просто берем косинус между двумя векторами авторов. Косинусное сходство лучше всего подходит, если имеется много многомерных признаков (https://i0.wp.com/neptune.ai/wp-content/uploads/Recommender-Systems-ML-Metrics-vs-Business-Metrics_10.png?resize=474%2C110&ssl=1)

  б) Сходство Жаккара (Jaccard similarity)
Сходство Жаккара - это размер пересечения, деленный на размер объединения двух наборов элементов https://i0.wp.com/neptune.ai/wp-content/uploads/Recommender-Systems-ML-Metrics-vs-Business-Metrics_18.png?resize=437%2C252&ssl=1

  в) Расстояние Эвклида (Euclidean distance)
Расстояние между двумя авторами, равно длине соединяющих их отрезков. Пространство предпочтений — это доступные элементы (соавторы), а оси — это авторы, с которыми уже было взаимодействие. На основе имеющихся взаимодействий мы ищем соавторов со схожими научными интересами. Чем меньше расстояние между двумя авторами, тем выше вероятность того, что они интересуются той же научной проблемой https://i0.wp.com/neptune.ai/wp-content/uploads/Recommender-Systems-ML-Metrics-vs-Business-Metrics_5.png?resize=554%2C86&ssl=1

  г) Коэффициент корреляции Пирсона(Pearson correlation coefficient (PCC))
PCC — это мера наклона линии, которая представляет отношение между двумя векторами авторов. Она может варьироваться от -1 до 1, 0 означает отсутствие линейной корреляции https://i0.wp.com/neptune.ai/wp-content/uploads/Recommender-Systems-ML-Metrics-vs-Business-Metrics_19.png?resize=535%2C141&ssl=1

  
  2. Предсказательные метрики

Предсказательные метрики касаются того, насколько рейтинги рекомендательных систем близки к рейтингам пользователей. Они являются хорошим выбором для небинарных задач. 

  а) MAE — это средняя величина различий между рекомендацией и соответствующей оценкой, которую очень легко интерпретировать. Обратите внимание, что он не наказывает за большие ошибки или выбросы и присваивает таким случаям равные веса с другими. Это означает, что MAE дает довольно целостное представление о точности рейтинга, а не штрафует за большие ошибки https://i0.wp.com/neptune.ai/wp-content/uploads/Recommender-Systems-ML-Metrics-vs-Business-Metrics_14.png?resize=378%2C119&ssl=1

  б) RMSE — это квадратичная метрика, которая также измеряет среднюю величину, но квадратный корень имеет значение. RMSE придает большой вес большим ошибкам. Это означает, что это более полезно, когда выбросы нежелательны https://i0.wp.com/neptune.ai/wp-content/uploads/Recommender-Systems-ML-Metrics-vs-Business-Metrics_20-1.png?resize=471%2C141&ssl=1

На практике как RMSE, так и MAE обычно проверяются для моделей совместных рекомендаций на наборе данных с перекрестной проверкой в ​​K-кратном порядке. Однако с точки зрения бизнеса важны не только самые высокие значения RMSE или MAE, но и показатели неточности, используемые для оценки.

  3. Классификационные метрики.
  
  Показатели классификации оценивают способность рекомендательных систем принимать решения. Они являются хорошим выбором для таких задач, как определение релевантных или нерелевантных продуктов для пользователя. Для метрик поддержки принятия решений точный рейтинг игнорируется, тогда как для методов, основанных на ранжировании, он имеет неявное влияние посредством ранжирования. 

  а) Precision@k

  Precision@k - доля топ k рекомендаций, которые релевантны пользователю

  $P = \frac{\text{# of  top  k  recommendations  that  are  relevant}}{\text{# of items that are recommended}}$

  б) Mar@K and Map@K

  Среднее значение отзыва при K (Mar@k) измеряет recall при k-й рекомендации. Mar@k учитывает порядок рекомендаций и наказывает правильные рекомендации в зависимости от порядка рекомендаций. Map@k и Mar@k идеально подходят для оценки упорядоченного списка рекомендаций. 

  в) Recall@k или HitRatio@k

  Recall@k или HitRatio@k — это часть k лучших рекомендуемых элементов, которые находятся в наборе элементов, релевантных пользователю. Чем больше k, тем выше коэффициент попаданий, поскольку выше вероятность того, что правильный ответ будет включен в рекомендации.
  
  $R = \frac{\text{(# of top k recommendations that are relevant}}{\text{# of all relevant items}}$

  г) Средняя точность (AP)
  В то время как precision@k (P(k)) учитывает только подмножество ваших рекомендаций от ранга 1 до k, средняя точность вознаграждает нас за размещение правильных рекомендаций в начале списка. Например, если precision@5 это константа, то AP@5 убывает с рангом рекомндованного объекта. Очень важно отметить, что AP не будет наказывать нас за включение дополнительных рекомендаций в наш список. При его использовании мы должны убедиться, что рекомендуем только лучшие объекты  https://i0.wp.com/neptune.ai/wp-content/uploads/Recommender-Systems-ML-Metrics-vs-Business-Metrics_27.png?resize=480%2C86&ssl=1






